**Analisis de la Encuesta Continua de Hogares 2024**

- Dataset: https://www4.ine.gub.uy/Anda5/index.php/catalog/767/get-microdata
- Diccionario: https://www4.ine.gub.uy/Anda5/index.php/catalog/767/data-dictionary/F4?file_name=ECH_implantacion_2024

In [ ]:
# CARGO EL CSV CON PANDAS
import pandas as pd
ech = pd.read_csv(r'C:\Users\ut603933\UM\tesis\ECH_implantacion_2024.csv')

: 

In [ ]:
# INFORMACION DEL DATASET
# print("Información del dataset de entrenamiento:")
# print(ech.info())
#print("\nResumen estadístico:")
#print(ech.describe())

In [ ]:
# VERIFICAR VALORES NULOS
# print("Columnas con valores nulos (por nombre de la variable):")
# print(ech.isnull().sum()[ech.isnull().sum() > 0].sort_index())
#print("Columnas con valores nulos (ordenadas):")
#print(ech.isnull().sum()[ech.isnull().sum() > 0].sort_values(ascending=False))

In [ ]:
# from tabulate import tabulate
# print(tabulate(ech, headers = 'keys', tablefmt = 'psql'))

In [ ]:
# MOSTRAR EL DF
# pd.set_option('display.max_rows', None)  # Show all rows
# pd.set_option('display.max_columns', 10)  # Show all columns
# pd.set_option('display.width', 100)  # Adjust display width
# pd.set_option('display.max_colwidth', None)  # Show full column contents
# print(ech)
# #pd.reset_option('all') # Restore default option

In [ ]:
# ech.sample(5)

In [ ]:
# IMPORTO LIBRERIAS
#import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
# === 1. Información general del dataset ===
print("Dimensiones del dataset:", ech.shape)
print("\nPrimeras filas:")
display(ech.head())

print("\nTipos de datos:")
display(ech.dtypes.value_counts())

In [ ]:
# Mostrar todas las columnas sin truncar
pd.set_option('display.max_columns', None)  # muestra todas las columnas
pd.set_option('display.expand_frame_repr', False)  # evita salto de línea

# Mostrar todos los nombres de columna
print(ech.columns.tolist())

In [ ]:
# Ver si hay columnas duplicadas por nombre
duplicated_cols = ech.columns[ech.columns.duplicated()].tolist()
print("Columnas duplicadas por nombre:", duplicated_cols)

No hay columnas duplicadas en el dataset original. Las columnas que aparecen duplicadas y no permiten ejecutar el pipeline las repetimos erroneamente al categorizar las variables producto de que si aparecen duplicadas en el Excel del diccionario.

## Analisis exploratorio de datos

In [ ]:
# === 2. Análisis de valores nulos ===
nulos = ech.isnull().sum()
nulos = nulos[nulos > 0].sort_values(ascending=False)
print("\nColumnas con nulos (cantidad):")
display(nulos)

Verificando el diccionario, los valores nulos son para las variables:
- c5_13: PROBLEMAS DE LA VIVIENDA Pisos y muros agrietados
- d181, d181_b, d184, d184_1, d184_b, d229, d230, d231, d232: SERVICIO DOMESTICO
- f288, f289, f290, f291_a, f291_b, f292, f293, f294, f295: Lugar y modalidad de trabajo

In [ ]:
nulos_pct = ech.isnull().mean().sort_values(ascending=False)
display(nulos_pct[nulos_pct > 0])

Las preguntas son de los modulos:
-   Servicio doméstico → sólo hogares que contratan personal.
-   Lugar y modalidad de trabajo → sólo personas ocupadas.

Confirmamos que son variables condicionales por diseño: 
-   La mitad de la población no responde esas preguntas porque no aplica.
-   El 50% restante sí tiene datos válidos y útiles.

No eliminamos esas columnas por tener “demasiados nulos”, ya que no se trata de datos ausentes por error, sino de lógica del cuestionario.

Podemos usarlas de forma segmentada, por ejemplo:
-   Hacer modelos distintos para personas ocupadas (que tienen datos en f288–f295) y no ocupadas, o
-   Incluir una variable indicadora de “aplica o no aplica” para evitar que el modelo se confunda.

Son una señal indirecta de pertenencia a ciertos grupos: tener datos en esas columnas podría implicar estar ocupado o tener personal a cargo → lo que sí puede estar correlacionado con el ingreso.

In [ ]:
# === 3. Distribución de la variable objetivo ===
# Suponemos que la variable objetivo es 'YDA'
target = 'YDA'
print("\nEstadísticas de ingreso (YDA):")
display(ech[target].describe())

plt.figure(figsize=(8,5))
sns.histplot(ech[target], bins=50, kde=True)
plt.title('Distribución del Ingreso Disponible Ajustado (YDA)')
plt.xlabel('Ingreso')
plt.ylabel('Frecuencia')
plt.show()

Se observa que:
- Muy asimétrica (sesgo a la derecha).
- Muchos ingresos entre $0 y $200.000.
- Algunos valores extremos que superan el millón (outliers evidentes).

Tratamos a continuacion: 
- Usmos la variable log_YDA como target en modelos lineales y árboles si queremos reducir la sensibilidad a los valores extremos.
- Visualizamos un boxplot ya que el max es 25 veces el 75º percentil.
- Vemos valores en 0 ya que min = 0 → ¿Ingreso nulo o inactivo?

In [ ]:
# target en modelos lineales y árboles si queremos reducir la sensibilidad a los valores extremos
ech['log_YDA'] = np.log1p(ech['YDA'])  # log(1 + YDA)

In [ ]:
# visualizar con boxplot y/o stripplot
sns.boxplot(x=ech['YDA'])
plt.title("Boxplot de ingreso (YDA)")
plt.show()

La gráfica muestra claramente lo que ya anticipamos:
- Una alta concentración de valores en el rango bajo (entre $0 y $200.000).
- Una larga cola derecha de outliers extremos, que se extienden hasta los 2.5 millones.

🔍 Esto confirma que:
- El ingreso en la ECH 2024 está fuertemente sesgado.
- La dispersión es muy alta.
- Los outliers son numerosos pero no necesariamente errores.

In [ ]:
# ver valores en cero
ech[ech['YDA'] == 0].shape

Esto significa que solo 12 personas tienen ingreso 0 declarado, lo cual:
- ✅ No es un problema de datos generalizado, ni requiere imputación.
- ⚠️ Pero sí vale la pena entender el perfil de estas 12 personas.

In [ ]:
display(ech[ech['YDA'] == 0][['e27', 'e26', 'f269', 'd8_1', 'd9', 'c1']])  # o las columnas que representen edad, ocupación, tipo de hogar

La mayoria inquilinos o arrendatarios (d8_1).

La distribución de YDA:
- Se comporta como es esperable en datos de ingreso.
- No requiere imputación de nulos en YDA (no hay).

✅ Conclusiones clave:
- Los valores extremos están presentes, pero no dominan.
- No es necesario eliminarlos todavía. Algunos modelos (como árboles) los manejan bien.

Transformación logarítmica es aconsejable.
Especialmente para modelos sensibles a la escala o distribución (Regresión Lineal, Redes Neuronales).

Pocos casos con ingreso cero.
Son casos especiales que podés dejar o filtrar si hacés segmentación (ej. "ocupados", "con ingreso distinto de cero", etc.).

Sí se beneficia de:
- Log-transformación (log1p).
- Revisión de outliers extremos.
- Segmentación posterior por tipo de población (ocupada/no ocupada).

In [ ]:
# === 4. Identificar variables numéricas y categóricas ===
num_vars = ech.select_dtypes(include=['int64', 'float64']).columns.tolist()
cat_vars = ech.select_dtypes(include=['object', 'category']).columns.tolist()

# Eliminar la variable target de ambas listas si está incluida
if target in num_vars: num_vars.remove(target)
if target in cat_vars: cat_vars.remove(target)

print(f"Variables numéricas: {len(num_vars)}")
print(f"Variables categóricas: {len(cat_vars)}")

Muchas variables categóricas están codificadas numéricamente, por ejemplo:
- sexo: 1 = Hombre, 2 = Mujer
- estado civil actual: 1 = Separado, 2 = Divorciado, 3 = Casado, etc.
- nivel que esta cursando: 4 = Educacion media basica, 6 = Educacion media superior, etc.

Estas se almacenan como int64 o float64, por lo que pandas las interpreta como numéricas, cuando en realidad son categóricas ordinales o nominales.

No imputamos variables numericas ni categoricas.

In [ ]:
# === 5. Tratamiento de valores nulos ===
# Para numéricas: imputar con la mediana
#for col in num_vars:
#    if ech[col].isnull().any():
#        ech[col] = ech[col].fillna(ech[col].median())

# Para categóricas: imputar con una categoría especial
#for col in cat_vars:
#    if ech[col].isnull().any():
#        ech[col] = ech[col].fillna('No informado')

Posible clasificacion de variables categoricas.

In [ ]:
# === 6. Codificación de variables categóricas ===
# Convertir binarias codificadas como 1/2 o Sí/No a 0/1
#def codificar_binarias(df):
#    for col in df.columns:
#        if df[col].nunique() == 2:
#            vals = sorted(df[col].dropna().unique())
#            if set(vals) <= set([0,1,2]):
#                df[col] = df[col].map({vals[0]: 0, vals[1]: 1})
#    return df

#ech = codificar_binarias(ech)

# One-hot encoding para variables categóricas no binarias
#ech_encoded = pd.get_dummies(ech, columns=cat_vars, drop_first=True)

Imputación condicional basada en la lógica del cuestionario:

Si PERSONAS PAGAS PARA REALIZAR TAREAS DOMÉSTICAS O DE CUIDADOS d181 (*) 1 = Sí / 2 = No es 2 = No entonces:
- PERSONAS CONTRATADAS PARA REALIZAR TAREAS  DOMÉSTICAS O DE CUIDADO d229 (*) N° Cantidad de personas contratadas es 0.
- CANTIDAD DE HORAS TRABAJADAS A LA SEMANA d230 (*) N° Cantidad de horas habitualmente trabajadas en la semana es 0.
- PERNOCTA EN EL HOGAR d231 (*) 1 = Sí / 2 = No es 2 = No.
- CANTIDAD DE PERSONAS QUE PERNOCTAN d232 (*) N° Cantidad de personas que pernoctan en el hogar es 0.

Si AYUDA GRATUITA DE OTROS FAMILIARES QUE NO INTEGRAN EL HOGAR d184 (*) 1 = Sí / 2 = No es 2 = No entonces:
- CANTIDAD DE HORAS TRABAJADAS A LA SEMANA d184_1 (*) N° Cantidad de horas habitualmente trabajadas en la semana es 0.

In [ ]:
# Si d181 == 2 (no tiene servicio doméstico), completar columnas relacionadas

# Estas deben ser 0 si están vacías
for col in ['d229', 'd230', 'd232']:
    ech.loc[(ech['d181'] == 2) & (ech[col].isnull()), col] = 0

# Esta debe ser 2 si está vacía (respuesta negativa explícita)
ech.loc[(ech['d181'] == 2) & (ech['d231'].isnull()), 'd231'] = 2

# Si d184 == 2 (no tiene ayuda gratuita), entonces d184_1 debe ser 0 si está vacía
ech.loc[(ech['d184'] == 2) & (ech['d184_1'].isnull()), 'd184_1'] = 0

In [ ]:
# Verificación de valores nulos después de imputación corregida
cols_verificar = ['d229', 'd230', 'd231', 'd232', 'd184_1']

print("Verificación de valores nulos en variables imputadas:")
print(ech[cols_verificar].isnull().sum())

Si f269 TRABAJO LA SEMANA ANTERIOR es 2 = No imputamos las siguientes variables tambien como 2 = No:
- f290 USO DE PC, TELÉFONO INTELIGENTE O TABLETA PARA TRABAJAR
- f291_a OTRO LUGAR DIFERENTE AL HOGAR, LOCAL PROPIO U OFICINA O INSTALACIÓN DEL CLIENTE
- f291_b TRABAJO EN UN LUGAR DIFERENTE AL HOGAR O AL LUGAR HABITUAL DE TRABAJO
- f292 TRABAJO REALIZADO FUERA DE SUS PROPIAS INSTALACIONES
- f295 TRABAJO SEMANAL EN 2 LUGARES DIFRENTES A LAS INSTALACIONES DEL EMPLEADOR/A O PROPIO LOCAL

In [ ]:
cols_f269 = ['f290', 'f291_a', 'f291_b', 'f292', 'f293', 'f295']
for col in cols_f269:
    ech.loc[(ech['f269'] == 2) & (ech[col].isnull()), col] = 2

Unificacion de variables d181 con d181_b y d184 con d184_b:

- d181 y d181_b miden lo mismo ("personas pagas para tareas domésticas o de cuidados"), pero en distintos semestres.
- d184 y d184_b también miden lo mismo ("ayuda gratuita de familiares fuera del hogar").
- Ambas parejas de variables tienen 50% de valores nulos complementarios, por lo que es totalmente válido consolidarlas en una nueva variable que conserve el significado original.

Para cada par, usamos el valor no nulo que exista.

Si los dos valores son nulos (por ejemplo, por no aplicar), la variable consolidada también quedará nula.

Primero recodificamos d181_b y d184_b a binario, ya que toman valores 1, 2 y 3 para Si y 1 y 2 para Si respectivamente.

In [ ]:
# Contamos cuántos ceros hay en d181 y d184 antes de imputar
n_d181_ceros = (ech['d181'] == 0).sum()
n_d184_ceros = (ech['d184'] == 0).sum()

print(f"Cantidad de ceros en d181: {n_d181_ceros}")
print(f"Cantidad de ceros en d184: {n_d184_ceros}")

Imputación de valores 0 en d181 y d184:

Durante el procesamiento inicial, se imputaron valores 0 en d181 y d184 en casos donde no se respondió la pregunta sobre servicio doméstico o ayuda gratuita.

Estos 0 representan casos no aplicables (por ejemplo, hogares sin servicio doméstico), por lo tanto, decidimos reemplazarlos por el valor 2 (No) antes de crear las columnas unificadas d181_unificado y d184_unificado.

Esto evita que la unificación resulte en valores faltantes injustificados y preserva la lógica del cuestionario.

In [ ]:
# Reemplazamos 0 por 2 (No aplica) en las variables d181 y d184 antes de unificarlas
ech.loc[ech['d181'] == 0, 'd181'] = 2
ech.loc[ech['d184'] == 0, 'd184'] = 2

In [ ]:
# Recodificar d181_b a binario (1 = Sí, 2 = No)
d181_b_bin = ech['d181_b'].map({1: 1, 2: 1, 3: 1, 4: 2})

# Unificar d181 y d181_b
ech['d181_unificado'] = ech['d181'].combine_first(d181_b_bin)

# Recodificar d184_b a binario (1 = Sí, 2 = No)
d184_b_bin = ech['d184_b'].map({1: 1, 2: 1, 3: 2})

# Unificar d184 y d184_b
ech['d184_unificado'] = ech['d184'].combine_first(d184_b_bin)

In [ ]:
# Código de validación para las columnas unificadas
# Validar que no haya valores nulos
print("Valores nulos en columnas unificadas:")
print(ech[['d181_unificado', 'd184_unificado']].isnull().sum())

# Validar que solo haya valores 1 o 2
print("\nValores únicos en d181_unificado:")
print(ech['d181_unificado'].value_counts(dropna=False).sort_index())

print("\nValores únicos en d184_unificado:")
print(ech['d184_unificado'].value_counts(dropna=False).sort_index())

# Validar proporción de respuestas "Sí" y "No"
print("\nProporción de respuestas (1 = Sí, 2 = No):")
print("d181_unificado:")
print(ech['d181_unificado'].value_counts(normalize=True).map(lambda x: f"{x:.2%}"))

print("d184_unificado:")
print(ech['d184_unificado'].value_counts(normalize=True).map(lambda x: f"{x:.2%}"))

Esto implica que solo 8 de cada 100 hogares pagan a una persona por tareas domésticas o cuidados. Esto es completamente razonable en el contexto uruguayo, dado que este tipo de contratación suele estar concentrada en hogares de mayor poder adquisitivo o con necesidades específicas de cuidado. Son coherentes con lo esperado para la población general en una encuesta nacional.

Menos del 5% de los hogares reciben ayuda gratuita de otros familiares que no integran el hogar. También es razonable. La solidaridad familiar informal existe, pero no es una práctica diaria ni generalizada como para que aparezca en un alto porcentaje en una encuesta estructurada.

Ambas proporciones:
- Son coherentes con lo esperado para la población general en una encuesta nacional.
- Validan que la lógica de unificación y recodificación fue correcta.
- No presentan valores anómalos o distribuciones sesgadas artificialmente (como hubiera pasado si imputaras todo como 2 sin justificación).

In [ ]:
# Eliminamos las columnas originales si ya no las necesitamos
ech.drop(columns=['d181', 'd181_b', 'd184', 'd184_b'], inplace=True)

In [ ]:
# === 7. Correlación con la variable objetivo (opcional visualización) ===
# Seleccionamos solo las columnas numéricas para evitar errores
ech_numericas = ech.select_dtypes(include=['number'])

# Calculamos correlaciones solo sobre esas columnas
correlaciones = ech_numericas.corr()[target].drop(target).sort_values(ascending=False)

# Mostramos resultados
print("\nTop 10 variables correlacionadas con el ingreso (YDA):")
display(correlaciones.head(10))

# Graficamos
plt.figure(figsize=(10,6))
sns.barplot(x=correlaciones.head(10).values, y=correlaciones.head(10).index)
plt.title("Top 10 variables correlacionadas con el ingreso")
plt.xlabel("Correlación")
plt.ylabel("Variable")
plt.tight_layout()
plt.show()

Verificando el diccionario, las variables correlacionadas con el ingreso son:
- HT11: INGRESO TOTAL DEL HOGAR CON VALOR LOCATIVO SIN SERVICIO DOMÉSTICO
- YDA_SVL: INGRESO DISPONIBLE AJUSTADO (SIN VALOR LOCATIVO)
- YSVL: INGRESO TOTAL DEL HOGAR SIN VALOR LOCATIVO SIN SERVICIO DOMÉSTICO
- d8_3: TENENCIA DE LA VIVIENDA > Monto del alquiler (efectivamente pagado o estimado)
- eg_ps2: MONTO MÍNIMO MENSUAL REQUERIDO PARA SATISFACER LAS NECESIDADES BÁSICAS
- HT13: VALOR LOCATIVO
- d14: CANTIDAD DE BAÑOS
- d21_15_4: ELEMENTOS DE CONFORT > Cantidad de microcomputadores que no son del Plan Ceibal
- PT1: TOTAL DE INGRESOS PERSONALES
- d21_14_1: ELEMENTOS DE CONFORT > Cantidad de equipos de aire acondicionado

Profundizamos en la correlacion con la variable objetivo:
- Usar correlación de Pearson para variables numéricas → detectar alta correlación con YDA o log_YDA.
- Usar feature importance rápida con un modelo tipo RandomForestRegressor para detectar también correlación no lineal o relaciones más complejas.
- Generar un listado de columnas candidatas a eliminar.

In [ ]:
# =========================
# BLOQUE 0 - Preparación
# =========================
df_corr_rf = ech.copy()  # Usa tu DataFrame original con las 535 variables
target_var = 'YDA'

# Eliminar filas donde el target sea NaN
df_corr_rf = df_corr_rf[df_corr_rf[target_var].notna()]

In [ ]:
# =========================
# BLOQUE 1 - Correlación
# =========================
# Selecciona variables numéricas reales (excluyendo el target)
num_vars = df_corr_rf.select_dtypes(include=[np.number]).columns.drop(target_var, errors='ignore')

correlaciones = df_corr_rf[num_vars].corrwith(df_corr_rf[target_var]).sort_values(key=abs, ascending=False)

print("\n📌 Correlación de Pearson con YDA (top 20):")
print(correlaciones.head(20))

# Variables con correlación alta (umbral configurable, ej. >= 0.85)
umbral_corr = 0.85
vars_corr_altas = correlaciones[correlaciones.abs() >= umbral_corr].index.tolist()

print(f"\nVariables con correlación |r| >= {umbral_corr}:")
print(vars_corr_altas)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# =========================
# BLOQUE 2 - Importancia con Random Forest
# =========================
if len(num_vars) > 0:
    print("\n📌 Entrenando Random Forest para evaluar importancia de variables...")
    rf = RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1)
    rf.fit(df_corr_rf[num_vars], df_corr_rf[target_var])

    importancias_rf = pd.Series(rf.feature_importances_, index=num_vars).sort_values(ascending=False)

    print("\n📌 Importancia según Random Forest (top 20):")
    print(importancias_rf.head(20))

    # Variables con importancia muy alta (ej. >= 0.05)
    umbral_importancia = 0.05
    vars_importantes_rf = importancias_rf[importancias_rf >= umbral_importancia].index.tolist()

    print(f"\nVariables con importancia >= {umbral_importancia}:")
    print(vars_importantes_rf)

else:
    print("\n⚠️ No hay variables numéricas para entrenar el Random Forest.")
    importancias_rf = pd.Series(dtype=float)
    vars_importantes_rf = []

In [ ]:
# =========================
# BLOQUE 3 - Unificación de listas
# =========================
vars_a_eliminar = list(set(vars_corr_altas + vars_importantes_rf))
print(f"\n📌 Total variables candidatas a eliminar por alta relación con {target_var}: {len(vars_a_eliminar)}")
print(vars_a_eliminar)

Preprocesamiento de variables categoricas dicotomicas, nominales u ordinales y numericas discretas o continuas. 

No incluimos:
- HT11: INGRESO TOTAL DEL HOGAR CON VALOR LOCATIVO SIN SERVICIO DOMÉSTICO
- YSVL: INGRESO TOTAL DEL HOGAR SIN VALOR LOCATIVO SIN SERVICIO DOMÉSTICO
- YDA_SVL: INGRESO DISPONIBLE AJUSTADO (SIN VALOR LOCATIVO)
- YDA: INGRESO DISPONIBLE AJUSTADO


In [ ]:
#import pandas as pd
#import numpy as np
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split

In [ ]:
# ====== PASO 1: Definir las listas de variables según nuestro Excel ======

dicotomicas = ['c5_2', 'c5_10', 'c5_11', 'c5_12', 'c5_13', 'c6', 'd8_4', 'd15', 'd21_1', 'd21_2', 'd21_3', 'd21_6', 'd21_4', 'd21_5', 'd21_20', 'd21_7', 'd21_10', 'd21_11', 'd21_12', 'd21_13', 'd21_14', 'd21_15', 'd21_15_1', 'd21_15_3', 'd21_15_5', 'd21_16', 'd21_16_1', 'd21_16_2', 'd21_21', 'd21_17', 'd21_18', 'd21_19', 'd181_unificado', 'd231', 'd184_unificado', 'e26', 'e29_1', 'e29_2', 'e29_3', 'e29_4', 'e29_5', 'e31', 'e32', 'e33', 'e185', 'e38', 'e45_4_1_cv', 'e45_cvb', 'e46_cv', 'e48', 'e581a', 'e582', 'e197_1', 'e201_1a', 'e201_1b', 'e201_1c', 'e201_1d', 'e51_6b', 'e215_1', 'e218_1', 'e221_1', 'e224_1', 'e559', 'e584', 'e59', 'f269', 'f270', 'f271', 'f272', 'f274', 'f276', 'f82', 'f84', 'f278', 'f280_1', 'f280_2', 'f280_3', 'f281_1', 'f281_2', 'f281_3', 'f281_4', 'f75', 'f81', 'f266_2', 'f267', 'f268', 'f289', 'f290', 'f291_a', 'f291_b', 'f292', 'f293', 'f294', 'f295', 'f96', 'f99', 'f100', 'f102', 'f298', 'f299', 'f114', 'f115', 'f300', 'f116', 'f117', 'f123', 'f124_1', 'f124_2', 'f124_3', 'f124_5', 'g250_1', 'g250_2', 'g250_3', 'g250_4', 'g250_5', 'g127', 'g128', 'g129', 'g129_1', 'g130', 'g131', 'g133', 'g_st_1', 'g251_1', 'g251_2', 'g251_3', 'g251_4', 'g251_5', 'g135', 'g136', 'g137', 'g137_1', 'g138', 'g139', 'g141', 'g_itnd_1', 'g142', 'g_itnd_2', 'g144', 'g_it_1', 'g_it_2', 'g149', 'g149_1', 'g150', 'g255', 'g152', 'g153', 'g258', 'g154', 'h155', 'h156', 'h272', 'h273', 'h274', 'h252', 'h159', 'h160', 'h161', 'h162', 'h227', 'h269', 'h167_2', 'h167_3', 'h167_4', 'h169', 'h271', 'h171', 'h172', 'h173', 'i228', 'i259', 'eg_ps1', 'SUBEMPLEO', 'pobre06', 'indig06', 'pobre_multi', 'pobre17', 'indig17']

nominales = ['dpto', 'nom_dpto', 'secc', 'ESTRED13', 'LOC_AGR_13', 'NOM_LOC_AGR_13', 'c1', 'd8_1', 'd11', 'd12', 'e557', 'e29_6', 'e30', 'e35', 'e36', 'e37', 'e234_2', 'e39', 'e235_2', 'e236', 'e236_4', 'e45_cv', 'e45_1_1_cv', 'e45_2_1_cv', 'e45_3_1_cv', 'e45_cva', 'e47_cv', 'e190', 'e49', 'e581', 'e209_1', 'e202', 'e214_1', 'e217_1', 'e220_1', 'e223_1', 'e226_1', 'e246', 'f273', 'f69', 'f69_1', 'f277', 'f71_2', 'f72_2', 'f73', 'f83', 'f278_a', 'f279', 'f76_2', 'f266', 'f266_1', 'f305', 'f306', 'f78', 'f80', 'f285', 'f286', 'f287', 'f288', 'f90_2', 'f91_2', 'f92', 'f97', 'f94', 'f101', 'f103', 'f104', 'f110', 'f111', 'f108', 'f301', 'f106', 'f122', 'f119_2', 'f120_2', 'f121', 'f125', 'g132', 'g140', 'g_itnd_3', 'g256', 'POBPCOAC', 'USO_RRAA']

ordinales = ['region', 'REGION_4', 'c2', 'c3', 'c4', 'd13', 'd16', 'd18', 'd260', 'd19', 'd20', 'e579', 'e583', 'e579a', 'f275', 'f283', 'f77', 'f93', 'h167_1', 'eg_ahorro', 'eg_ps3', 'eg_ps4', 'eg_ps5', 'eg_ps6', 'eg_ps7', 'eg_ps8']

numericas_discretas = ['ID', 'nper', 'anio', 'mes', 'GR', 'ccz', 'barrio', 'c6_1', 'd9', 'd10', 'd14', 'd21_4_1', 'd21_5_1', 'd21_14_1', 'd21_15_2', 'd21_15_4', 'd21_15_6', 'd21_18_1', 'd21_19_1', 'd229', 'd230', 'd232', 'd184_1', 'd23', 'd24', 'd25', 'e27', 'e31_1', 'e32_1', 'e34', 'e186_1', 'e186_2', 'e186_3', 'e186_4', 'e37_2', 'e38_1', 'e39_2', 'e236_2', 'e45_1_1_1_cv', 'e45_2_1_1_cv', 'e45_3_1_1_cv', 'e45_4_1_1_cv', 'e47_1_cv', 'e49a', 'e582_1', 'e582_2', 'e582_3', 'e51_2', 'e51_3', 'e51_4_a', 'e51_4_b', 'e51_5', 'e51_6', 'e51_6a', 'e51_8', 'e51_9', 'e51_10', 'e51_11', 'e559_1', 'e559_2', 'e247', 'f70', 'f80_2', 'f284_1', 'f284_2', 'f284_3', 'f284_4', 'f284_5', 'f284_6', 'f284_7', 'f85', 'f307', 'f308', 'f94_2', 'f296_1', 'f296_2', 'f296_3', 'f296_4', 'f296_5', 'f296_6', 'f296_7', 'f98', 'f297', 'f113', 'f118_1', 'f118_2', 'g127_1', 'g127_2', 'g132_1', 'g132_2', 'g132_3', 'g135_1', 'g135_2', 'g140_1', 'g140_2', 'g140_3', 'g151_6', 'g151_3', 'g151_4', 'h272_1', 'h273_1', 'h274_1', 'h274_2', 'h274_3', 'h158_1', 'h158_2', 'h171_2', 'HT19']

numericas_continuas = ['d8_2', 'd8_3', 'e584_1', 'g_id_1', 'g_id_2', 'g_id_3', 'g_id_1a', 'g_id_2a', 'g_id_3a', 'g126_1', 'g126_2', 'g126_3', 'g126_4', 'g126_5', 'g126_6', 'g126_7', 'g126_8', 'g127_3', 'g128_1', 'g129_2', 'g130_1', 'g131_1', 'g133_1', 'g133_2', 'g134_1', 'g134_2', 'g134_3', 'g134_4', 'g134_5', 'g134_6', 'g134_7', 'g134_8', 'g135_3', 'g136_1', 'g137_2', 'g138_1', 'g139_1', 'g141_1', 'g141_2', 'g143', 'g144_1', 'g144_2_1', 'g144_2_2', 'g144_2_3', 'g144_2_4', 'g144_2_5', 'g259', 'g148_1_1', 'g148_1_2', 'g148_1_3', 'g148_1_5', 'g148_1_6', 'g148_1_7', 'g148_1_8', 'g148_1_9', 'g148_1_10', 'g148_1_11', 'g148_1_12', 'g148_2_1', 'g148_2_2', 'g148_2_3', 'g148_2_5', 'g148_2_6', 'g148_2_7', 'g148_2_8', 'g148_2_9', 'g148_2_10', 'g148_2_11', 'g148_2_12', 'g148_3', 'g148_4', 'g148_5_1', 'g148_5_2', 'g257', 'g153_1', 'g153_2', 'g258_1', 'g154_1', 'h155_1', 'h156_1', 'h252_1', 'h160_1', 'h160_2', 'h163_1', 'h163_2', 'h164', 'h165', 'h166', 'h269_1', 'h167_1_3', 'h167_2_3', 'h167_3_3', 'h167_4_3', 'h170_3', 'h271_1', 'h171_1', 'h172_1', 'h173_1', 'i174', 'i175', 'eg_ps2', 'MTO_CUOTA', 'MTO_EMER', 'MTO_HOGCON', 'MTO_DESAY', 'MTO_ALMUE', 'MTO_VACAS', 'MTO_OVEJA', 'MTO_CABALL', 'INDACELIAC', 'INDAEMER', 'PT1', 'PT2', 'PT4', 'HT13', 'YHOG', 'AFAM_H_DEC', 'AFAM_H', 'TUS_H_DEC', 'TUS_H', 'lp_06', 'li_06', 'lp_17', 'li_17', 'monto_imput_UTE', 'monto_imput_GAS', 'monto_imput_OSE', 'H_FONASA', 'montoGAS_RRAA', 'montoUTE_RRAA', 'montoOSE_RRAA', 'W_TRI', 'W_SEM', 'W_ANO']

# Variable objetivo
target = 'YDA'
ech['log_YDA'] = np.log1p(ech[target])

No se incluyeron las siguientes variables porque si bien aparecen en el Excel no aparecen en el dataframe:

Dicotomicas:
- h2_cv: HOGAR CONTINÚA RESIDIENDO EN LA MISMA VIVIENDA
- c5_1, c5_3 a c5_9: PROBLEMAS DE LA VIVIENDA
- e0_cv: NUEVO MIEMBRO DEL HOGAR
- e01_cv: NUEVO MIEMBRO DEPENDE DEL FONDO DE ALIMENTACIÓN
- e1_cv: MIEMBRO CONTINÚA RESIDIENDO EN EL HOGAR
- f304: MANTIENE MISMO TRABAJO PRINCIPAL DECLARADO EN ENTREVISTA ANTERIOR
- f302: REALIZA LAS MISMAS TAREAS
- f303: EL ESTABLECIMIENTO CONTINUA REALIZANDO LAS MISMAS TAREAS
- f311: MANTIENE MISMO TRABAJO SECUNDARIO DECLARADO EN ENTREVISTA ANTERIOR
- f309: REALIZA LAS MISMAS TAREAS
- f310: EL ESTABLECIMIENTO CONTINUA REALIZANDO LAS MISMAS TAREAS
- INFORMAL: TRABAJADOR INFORMAL

Algunas variables toman valores nan luego de recodificar como variables dicotomicas. Vemos que valores toman.

El problema es que en el dataset hay valores que no son ni 1 ni 2 en algunas variables dicotómicas y cualquier valor diferente de 1 o 2 (por ejemplo, NaN, 9, 3, o cualquier otro código especial) se transforma automáticamente en NaN.

In [ ]:
variables_a_explorar = ['c5_2', 'c5_10', 'c5_11', 'c5_12', 'c5_13', 'c6', 'd8_4', 'd15', 'd21_1', 'd21_2', 'd21_3', 'd21_6', 'd21_4', 'd21_5', 'd21_20', 'd21_7', 'd21_10', 'd21_11', 'd21_12', 'd21_13', 'd21_14', 'd21_15', 'd21_15_1', 'd21_15_3', 'd21_15_5', 'd21_16', 'd21_16_1', 'd21_16_2', 'd21_21', 'd21_17', 'd21_18', 'd21_19', 'd181_unificado', 'd231', 'd184_unificado', 'h155', 'h156', 'h272', 'h273', 'h274', 'h252', 'h159', 'h160', 'h161', 'h162', 'h227', 'h269', 'h167_2', 'h167_3', 'h167_4', 'h169', 'h271', 'h171', 'h172', 'h173', 'i228', 'i259', 'eg_ps1', 'pobre06', 'indig06', 'pobre17', 'indig17', 'pobre_multi', 'e26', 'e29_1', 'e29_2', 'e29_3', 'e29_4', 'e29_5', 'e31', 'e32', 'e33', 'e185', 'e38', 'e45_4_1_cv', 'e45_cvb', 'e46_cv', 'e48', 'e581a', 'e582', 'e197_1', 'e201_1b', 'e201_1c', 'e201_1d', 'e51_6b', 'e215_1', 'e218_1', 'e221_1', 'e224_1', 'e559', 'e584', 'e59', 'f269', 'f270', 'f271', 'f272', 'f274', 'f276', 'f82', 'f84', 'f278', 'f280_1', 'f280_2', 'f280_3', 'f281_1', 'f281_2', 'f281_3', 'f281_4', 'f75', 'f81', 'f266_2', 'f267', 'f268', 'f289', 'f290', 'f291_a', 'f291_b', 'f292', 'f293', 'f294', 'f295', 'f96', 'f99', 'f100', 'f102', 'f298', 'f299', 'f114', 'f115', 'f300', 'f116', 'f117', 'f123', 'f124_1', 'f124_2', 'f124_3', 'f124_5', 'g250_1', 'g250_2', 'g250_5', 'g250_3', 'g250_4', 'g127', 'g128', 'g129', 'g129_1', 'g130', 'g131', 'g133', 'g_st_1', 'g251_1', 'g251_2', 'g251_5', 'g251_3', 'g251_4', 'g135', 'g136', 'g137', 'g137_1', 'g138', 'g139', 'g141', 'g_itnd_1', 'g_itnd_2', 'g144', 'g_it_1', 'g_it_2', 'g149', 'g149_1', 'g150', 'g255', 'g152', 'g153', 'g258', 'g154', 'SUBEMPLEO', 'indig06', 'pobre17', 'indig17']

for col in variables_a_explorar:
    if col in ech.columns:
        print(f"\nValores únicos en '{col}':")
        print(ech[col].unique())
    else:
        print(f"\nLa columna '{col}' no está en el DataFrame.")

In [ ]:
# Re-codificación de variables dicotómicas (1 → 1, 2 → 0, 0 → 0, NaN → 0)
for col in dicotomicas:
    if col in ech.columns:
        ech[col] = ech[col].map({1: 1, 2: 0, 0: 0, np.nan: 0}).fillna(0)
    else:
        print(f"Advertencia: {col} no está en el DataFrame y se omite.")

# Verificacion: Deberíamos ver solo [0, 1] o [1] (si la variable es constante).
for col in dicotomicas:
    print(f"{col}: {ech[col].unique()}")

No se incluyeron las siguientes variables porque si bien aparecen en el Excel no aparecen en el dataframe:

Nominal:
- h4_1_cv: DEPARTAMENTO DEL NUEVO DOMICILIO
- SIT_OCUP: SITUACIÓN EN LA OCUPACIÓN
- SECTOR_F: TIPO DE SECTOR

Ordinales: NIV_EDU: NIVEL EDUCATIVO

Discretas: ronda: ?

Continuas: W: PONDERADOR MENSUAL

El preprocesamiento original no aplicaba imputacion para valores faltantes por lo que al correr los modelos daba error.

Comentamos el preprocesamiento original y corremos un preprocesamiento mas robusto.

In [ ]:
# ====== PASO 2: Preprocesamiento ====== 

# Codificación de ordinales – si los valores están bien ordenados
#ordinal_encoder = OrdinalEncoder()

# Escalador para continuas
#scaler = StandardScaler()

# Codificador para nominales
#onehot = OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore')

# ColumnTransformer para aplicar todo junto
#preprocessor = ColumnTransformer(transformers=[
#    ('nom', onehot, nominales),
#    ('ord', ordinal_encoder, ordinales),
#    ('scale', scaler, numericas_continuas)
#], remainder='passthrough')  # deja discretas y dicotómicas sin tocar

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OrdinalEncoder, OneHotEncoder
from sklearn.impute import SimpleImputer

In [ ]:
# Pipelines individuales con imputación
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

ordinal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OrdinalEncoder(handle_unknown='use_encoded_value', unknown_value=-1))
])

nominal_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(drop='first', sparse_output=False, handle_unknown='ignore'))
])

binary_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent'))  # No codifica, ya vienen como 0/1
])

In [ ]:
# ColumnTransformer final
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, numericas_continuas),
    ('ord', ordinal_transformer, ordinales),
    ('nom', nominal_transformer, nominales),
    ('bin', binary_transformer, dicotomicas),
    ('disc', numeric_transformer, numericas_discretas)  # puede usar el mismo pipeline que numéricas continuas
])

In [ ]:
# ====== PASO 3: Separar X e y ======

#X = ech[dicotomicas + nominales + ordinales + numericas_discretas + numericas_continuas]

# Eliminar variables con fuga de información
#vars_a_eliminar = ['YDA', 'YDA_SVL']  # ajustar según auditoría
#X = X.drop(columns=[col for col in vars_a_eliminar if col in X.columns])

#y = ech['log_YDA']

In [ ]:
# 2) Separar features y target
X = ech[dicotomicas + nominales + ordinales + numericas_discretas + numericas_continuas]
y = ech['log_YDA']

Detectamos que los modelos que corrimos mas abajo sobreajustaban. Esto ocurria porque teniamos la variable num__YDA (generada en el pipeline) que aporta el 99.8% de la importancia del modelo segun un feature importance. Por lo tanto eliminamos las variables sospechosas antes del split y del fit del preprocesador y nos aseguramos de que:
- Nunca se usen estas variables en el entrenamiento.
- El pipeline y las métricas se calculen solo con variables legítimas.

In [ ]:
# 3) Filtrar listas de variables para que no incluyan columnas eliminadas
#dicotomicas = [c for c in dicotomicas if c not in vars_a_eliminar]
#nominales = [c for c in nominales if c not in vars_a_eliminar]
#ordinales = [c for c in ordinales if c not in vars_a_eliminar]
#numericas_discretas = [c for c in numericas_discretas if c not in vars_a_eliminar]
#numericas_continuas = [c for c in numericas_continuas if c not in vars_a_eliminar]

In [ ]:
# 3) Train-test split UNA sola vez
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Ver columnas duplicadas en X_train
from collections import Counter

contador_columnas = Counter(X_train.columns)
columnas_duplicadas = [col for col, count in contador_columnas.items() if count > 1]

print("Columnas duplicadas:", columnas_duplicadas)

In [ ]:
# Ver variables en comun entre los conjuntos
sets = {
    "ordinales": set(ordinales),
    "nominales": set(nominales),
    "numericas_discretas": set(numericas_discretas),
    "numericas_continuas": set(numericas_continuas),
    "dicotomicas": set(dicotomicas)
}

for nombre1, set1 in sets.items():
    for nombre2, set2 in sets.items():
        if nombre1 != nombre2:
            interseccion = set1 & set2
            if interseccion:
                print(f"Variables en común entre {nombre1} y {nombre2}: {interseccion}")

El pipeline final dio error porque algunas variables estaban repetidas porque se trajeron del Excel que las repetia.

In [ ]:
# ====== PASO 5: Pipeline final ======

#pipeline = Pipeline(steps=[
#    ('preprocesamiento', preprocessor)
#])

# Fit-transform solo para ver el shape de salida
#X_train_proc = pipeline.fit_transform(X_train)
#X_test_proc = pipeline.transform(X_test)
#print("Shape del dataset procesado:", X_train_proc.shape)

Dataset original (ech) con 55.923 filas, entonces con un test_size=0.2:
- X_train y y_train → 80% de los datos ≈ 44.738 filas
- X_test y y_test → 20% de los datos ≈ 11.185 filas

Coincide con Shape del dataset procesado: (44738, 2889):
- Las 44.738 filas son del conjunto de entrenamiento.
- Las 2.889 columnas son el resultado del preprocesamiento, principalmente por el OneHotEncoder

In [ ]:
# Asegurarse de que no estén en ninguna lista usada en preprocessor
for lista in [numericas_continuas, numericas_discretas, ordinales, nominales, dicotomicas]:
    if 'YDA' in lista:
        lista.remove('YDA')
    if 'YDA_SVL' in lista:
        lista.remove('YDA_SVL')

In [ ]:
# 4) Fit-transform SOLO con train y transform test
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc  = preprocessor.transform(X_test)

print("Shapes tras preprocesamiento:")
print("X_train_proc:", X_train_proc.shape)
print("X_test_proc :", X_test_proc.shape)

In [ ]:
# Contar cuántas filas tienen al menos un NaN
n_filas_nan = np.isnan(X_train_proc).sum(axis=1)
n_filas_con_nan = np.sum(n_filas_nan > 0)

print(f"Número de filas con al menos un NaN: {n_filas_con_nan}")

In [ ]:
# Contar columnas con NaN
n_columnas_con_nan = np.sum(np.isnan(X_train_proc).any(axis=0))
print(f"Número de columnas con al menos un NaN: {n_columnas_con_nan}")

Eliminamos redundancias automáticas (varianza cero y correlación perfecta).

In [ ]:
# Convertir a DataFrame con nombres de columnas
X_train_proc = pd.DataFrame(
    X_train_proc,
    columns=preprocessor.get_feature_names_out()
)
X_test_proc = pd.DataFrame(
    X_test_proc,
    columns=preprocessor.get_feature_names_out()
)

print(f"✅ Shape inicial tras preprocesamiento: {X_train_proc.shape}")

In [ ]:
# ============================================
# 2️⃣ Funciones de limpieza
# ============================================
def eliminar_varianza_cero(df):
    from sklearn.feature_selection import VarianceThreshold
    selector = VarianceThreshold(threshold=0.0)
    selector.fit(df)
    cols_quitar = df.columns[~selector.get_support()].tolist()
    df_filtrado = df.drop(columns=cols_quitar)
    return df_filtrado, cols_quitar

def eliminar_correlacion_perfecta(df):
    corr_matrix = df.corr()
    cols_quitar = set()
    for i in range(len(corr_matrix.columns)):
        for j in range(i):
            if corr_matrix.iloc[i, j] == 1:
                colname = corr_matrix.columns[i]
                cols_quitar.add(colname)
    df_filtrado = df.drop(columns=list(cols_quitar))
    return df_filtrado, list(cols_quitar)

def eliminar_correlacion_alta(df, umbral=0.95):
    corr_matrix = df.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    cols_quitar = [column for column in upper.columns if any(upper[column] > umbral)]
    df_filtrado = df.drop(columns=cols_quitar)
    return df_filtrado, cols_quitar

In [ ]:
# ============================================
# 3️⃣ Aplicar limpieza a TRAIN y TEST
# ============================================
# Varianza cero
X_train_proc, cols_var0 = eliminar_varianza_cero(X_train_proc)
X_test_proc = X_test_proc.drop(columns=cols_var0, errors='ignore')

# Correlación perfecta
X_train_proc, cols_corr1 = eliminar_correlacion_perfecta(X_train_proc)
X_test_proc = X_test_proc.drop(columns=cols_corr1, errors='ignore')

# Correlación alta
X_train_proc, cols_corr_high = eliminar_correlacion_alta(X_train_proc, umbral=0.95)
X_test_proc = X_test_proc.drop(columns=cols_corr_high, errors='ignore')

print(f"📉 Shape final tras limpieza: {X_train_proc.shape}")

In [ ]:
# ============================================
# 4️⃣ Guardar columnas eliminadas para trazabilidad
# ============================================
df_eliminadas = pd.DataFrame({
    "varianza_cero": pd.Series(cols_var0),
    "corr_perfecta": pd.Series(cols_corr1),
    "corr_alta": pd.Series(cols_corr_high)
})
df_eliminadas.to_excel("columnas_eliminadas.xlsx", index=False)
print("📂 Archivo 'columnas_eliminadas.xlsx' guardado.")

In [ ]:
hasta aca

## Entrenamiento

Comentamos los modelos iniciales con MSE.

In [ ]:
# Importo librerias
#from sklearn.linear_model import LinearRegression, Ridge, Lasso
#from sklearn.tree import DecisionTreeRegressor
#from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
#from sklearn.model_selection import cross_val_score
#from sklearn.metrics import make_scorer, mean_squared_error
#import numpy as np
#import pandas as pd

In [ ]:
# Importar XGBoost y LightGBM si están disponibles
#try:
#    from xgboost import XGBRegressor
#except ImportError:
#    XGBRegressor = None

#try:
#    from lightgbm import LGBMRegressor
#except ImportError:
#    LGBMRegressor = None

In [ ]:
# Definir modelos a evaluar
#modelos = {
#    "LinearRegression": LinearRegression(),
#    "Ridge": Ridge(),
#    "Lasso": Lasso(),
#    "DecisionTree": DecisionTreeRegressor(),
#    "RandomForest": RandomForestRegressor(n_jobs=-1),
#    "GradientBoosting": GradientBoostingRegressor()
#}

In [ ]:
# Agregar XGBoost si está disponible
#if XGBRegressor:
#    modelos["XGBoost"] = XGBRegressor(n_jobs=-1)

# Agregar LightGBM si está disponible
#if LGBMRegressor:
#    modelos["LightGBM"] = LGBMRegressor(n_jobs=-1)

In [ ]:
print("¿Hay NaN en X_train_proc?", np.isnan(X_train_proc).any())

In [ ]:
# Métrica: MSE negativo (lo convierte a positivo más abajo)
#scoring = make_scorer(mean_squared_error, greater_is_better=False)

In [ ]:
# Ejecuta validación cruzada y muestra resultados
#resultados = {}
#for nombre, modelo in modelos.items():
#    scores = cross_val_score(modelo, X_train_proc, y_train, cv=5, scoring=scoring)
#    mse_promedio = -np.mean(scores)
#    print(f"{nombre}: MSE promedio = {mse_promedio:,.2f}")
#    resultados[nombre] = mse_promedio

In [ ]:
# Mostrar resultados ordenados
#df_resultados = pd.DataFrame(resultados).T.sort_values(by='RMSE promedio')
#print("\nComparación de modelos por RMSE (menor es mejor):")
#display(df_resultados)

Entrenar modelos usando X_train_proc / y_train y evaluar en X_test_proc / y_test.

In [ ]:
# 0) Importo librerias
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

Primera funcion para evaluar modelo (la comentamos para no correrla).

In [ ]:
# Utilidad para evaluar en ambas escalas
#def evaluar_modelo(modelo, Xtr, ytr, Xte, yte, nombre="Modelo"):
    # Entrenar
#    modelo.fit(Xtr, ytr)
    # Predecir
#    y_pred = modelo.predict(Xte)

    # Métricas en escala log (si y es log_YDA)
#    mae = mean_absolute_error(yte, y_pred)
#    mse = mean_squared_error(yte, y_pred)
#    rmse = np.sqrt(mse)
#    r2 = r2_score(yte, y_pred)

#    print(f"\n=== {nombre} | Métricas en escala log (log_YDA) ===")
#    print(f"MAE : {mae:,.4f}")
#    print(f"MSE : {mse:,.4f}")
#    print(f"RMSE: {rmse:,.4f}")
#    print(f"R²  : {r2:,.4f}")

    # Métricas en escala original (deslogarizando)
#    yte_orig = np.expm1(yte)
#    y_pred_orig = np.expm1(y_pred)

#    mae_o = mean_absolute_error(yte_orig, y_pred_orig)
#    mse_o = mean_squared_error(yte_orig, y_pred_orig)
#    rmse_o = np.sqrt(mse_o)
#    r2_o = r2_score(yte_orig, y_pred_orig)

#    print(f"\n=== {nombre} | Métricas en escala original (pesos) ===")
#    print(f"MAE : {mae_o:,.2f}")
#    print(f"MSE : {mse_o:,.2f}")
#    print(f"RMSE: {rmse_o:,.2f}")
#    print(f"R²  : {r2_o:,.4f}")

#    return {
#        "mae_log": mae, "mse_log": mse, "rmse_log": rmse, "r2_log": r2,
#        "mae": mae_o, "mse": mse_o, "rmse": rmse_o, "r2": r2_o,
#        "y_pred": y_pred, "y_pred_orig": y_pred_orig
#    }

Funcion correjida con validacion defensiva (porque la regresion lineal daba error con la funcion anterior)

In [ ]:
def evaluar_modelo(modelo, X_train, y_train, X_test, y_test, nombre="modelo"):
    print("🟢 Entrenando modelo...")
    modelo.fit(X_train, y_train)

    print("🟢 Generando predicciones...")
    y_pred_train = modelo.predict(X_train)
    y_pred_test = modelo.predict(X_test)

    print("🔍 Verificando predicciones...")
    print(f"Máximo y_pred_test: {np.max(y_pred_test)}")
    print(f"Mínimo y_pred_test: {np.min(y_pred_test)}")

    print("📊 Calculando métricas en escala log...")
    r2 = r2_score(y_test, y_pred_test)
    mae = mean_absolute_error(y_test, y_pred_test)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred_test))

    # 🔁 Revertimos a escala de pesos, evitando overflow en np.exp
    y_test_clip = np.clip(y_test, 0, 30)
    y_pred_clip = np.clip(y_pred_test, 0, 30)

    y_test_original = np.exp(y_test_clip)
    y_pred_original = np.exp(y_pred_clip)

    print(f"Máximo y_pred_original (clipped): {np.max(y_pred_original)}")

    print("📊 Calculando métricas en pesos...")
    mae_pesos = mean_absolute_error(y_test_original, y_pred_original)
    rmse_pesos = np.sqrt(mean_squared_error(y_test_original, y_pred_original))

    print("✅ Métricas finales:")
    print(f"Modelo: {nombre}")
    print(f"R2: {r2:.4f} | MAE_log: {mae:.2f} | RMSE_log: {rmse:.2f}")
    print(f"MAE_pesos: {mae_pesos:.2f} | RMSE_pesos: {rmse_pesos:.2f}")

    return {
        "modelo": nombre,
        "R2": r2,
        "MAE_log": mae,
        "RMSE_log": rmse,
        "MAE_pesos": mae_pesos,
        "RMSE_pesos": rmse_pesos
    }

La funcion original daba error porque etabamos usando np.exp(y_pred_test) para volver a la escala original (en pesos), pero las predicciones incluyen valores extremadamente grandes. Eso explota al aplicar np.exp() y da como resultado inf, lo que rompe las métricas como mean_squared_error. Entonces limitamos las predicciones antes de aplicar np.exp() con np.clip(). Esto evitará que np.exp() explote al exponenciar números muy altos.

Esta estrategia es valida cuando:
- Estás evaluando predicciones en log-transformed regression.
- El modelo produce algunos valores desproporcionados (outliers extremos).
- El objetivo es evitar que un exp() distorsione totalmente las métricas.

Además, ya tenés tu variable objetivo (log_YDA) log-transformada, y no estás usando estos valores para reentrenamiento, sino para evaluación.

In [ ]:
# 1) Split en crudo (sin preprocesar aún)

# X = ech[dicotomicas + nominales + ordinales + numericas_discretas + numericas_continuas]
# y = ech['log_YDA']   # ya definido previamente

#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [ ]:
# 2) Ajustar SOLO con TRAIN y transformar TRAIN y TEST:
# El preprocessor es el ColumnTransformer definido antes.

#preprocessor.fit(X_train)

#X_train_proc = preprocessor.transform(X_train)
#X_test_proc  = preprocessor.transform(X_test)

#print("Shapes tras preprocesamiento:")
#print("X_train_proc:", X_train_proc.shape)
#print("X_test_proc :", X_test_proc.shape)

In [ ]:
# Verificamos valores invalidos
def verificar_valores_invalidos(X, nombre):
    print(f"🔎 Verificando {nombre}:")
    print("NaN:", np.isnan(X).sum())
    print("Inf:", np.isinf(X).sum())
    print("Valores demasiado grandes:", np.sum(np.abs(X) > 1e10))

verificar_valores_invalidos(X_train_proc, "X_train_proc")
verificar_valores_invalidos(X_test_proc, "X_test_proc")

No hay problemas en los conjuntos de X. Vemos y.

In [ ]:
print("🔎 y_train")
print("NaN:", np.isnan(y_train).sum())
print("Inf:", np.isinf(y_train).sum())
print("Valores extremos:", np.sum(np.abs(y_train) > 1e10))

print("\n🔎 y_test")
print("NaN:", np.isnan(y_test).sum())
print("Inf:", np.isinf(y_test).sum())
print("Valores extremos:", np.sum(np.abs(y_test) > 1e10))

In [ ]:
print("y_train - min:", np.min(y_train), "max:", np.max(y_train))
print("y_test - min:", np.min(y_test), "max:", np.max(y_test))

In [ ]:
varianzas = np.var(X_train_proc, axis=0)
print("Columnas con varianza cero:", np.sum(varianzas == 0))

In [ ]:
print("Valor máximo absoluto en X_train_proc:", np.max(np.abs(X_train_proc)))

In [ ]:
modelo_lr = LinearRegression()

try:
    modelo_lr.fit(X_train_proc, y_train)
    print("✅ Fit correcto")
except Exception as e:
    print("❌ Error durante el fit:", e)

try:
    y_pred = modelo_lr.predict(X_test_proc)
    print("✅ Predict correcto")
except Exception as e:
    print("❌ Error durante el predict:", e)

In [ ]:
# 3) Modelo 1: Regresión Lineal

resultados_modelos = []

modelo_lr = LinearRegression()
res_lr = evaluar_modelo(modelo_lr, X_train_proc, y_train, X_test_proc, y_test, nombre="Linear Regression")

# Verificación defensiva antes de agregar
if all(np.isfinite(v) for k, v in res_lr.items() if k != "modelo"):
    resultados_modelos.append(res_lr)
else:
    print("❌ Resultado no agregado por valores inválidos.")

Esos resultados me confirman que la regresión lineal sigue completamente inestable, incluso después de las modificaciones.
- R² negativo y enorme: Un R² muy negativo indica que el modelo no solo no está explicando nada, sino que lo está haciendo peor que una media constante. En un modelo estable y bien especificado, aunque sea malo, deberíamos ver R² cercano a 0, no un valor tan extremo.
- Predicciones extremas: Máximos y mínimos de y_pred_test muy alejados de lo esperado (miles de millones en log transformado) son síntoma de multicolinealidad o coeficientes enormes. Esto pasa porque la regresión lineal estándar no tiene regularización, y si hay correlación fuerte entre variables o un número de features muy grande, los coeficientes explotan.
- MAE y RMSE gigantes en pesos: Más de 50 mil millones de pesos de MAE y 735 mil millones de RMSE indican que el modelo no es usable en producción. Esto lo vimos antes: el comportamiento típico de un modelo que está sobreajustado a ruido y no generaliza.

Hacemos un VIF (Variance Inflation Factor) de las variables procesadas para detectar multicolinealidad.

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

In [ ]:
# =========================================
# 1) Calcular VIF en datos procesados
# =========================================

# X_train_proc es un array numpy, así que lo pasamos a DataFrame
# Usamos las columnas generadas por el preprocessor
feature_names = preprocessor.get_feature_names_out()
X_train_df = pd.DataFrame(X_train_proc, columns=feature_names)

# Calculamos VIF
vif_data = pd.DataFrame()
vif_data["feature"] = X_train_df.columns
vif_data["VIF"] = [
    variance_inflation_factor(X_train_df.values, i) for i in range(X_train_df.shape[1])
]

# Ordenamos por VIF descendente
vif_data = vif_data.sort_values(by="VIF", ascending=False)
print("\n📊 Top 20 features con mayor VIF:")
print(vif_data.head(20))

📊 Top 20 features con mayor VIF:
              feature  VIF
1442  nom__f72_2_8413  inf
1409  nom__f72_2_7210  inf
1423  nom__f72_2_7830  inf
1422  nom__f72_2_7820  inf
1421  nom__f72_2_7810  inf
1420  nom__f72_2_7730  inf
1419  nom__f72_2_7729  inf
1418  nom__f72_2_7721  inf
1417  nom__f72_2_7710  inf
1416  nom__f72_2_7500  inf
1415  nom__f72_2_7490  inf
1414  nom__f72_2_7420  inf
1413  nom__f72_2_7410  inf
1412  nom__f72_2_7320  inf
1411  nom__f72_2_7310  inf
1410  nom__f72_2_7220  inf
1408  nom__f72_2_7120  inf
1590      nom__f305_2  inf
1407  nom__f72_2_7110  inf
1406  nom__f72_2_7020  inf

Esos resultados son una señal clarísima de que tienes multicolinealidad perfecta en muchas variables, y además concentrada en un bloque muy específico: las columnas nom__f72_2_XXXX y alguna como nom__f305_2.

Que el VIF sea inf significa que alguna de esas variables es combinación lineal exacta de otras, lo cual es típico cuando:

Hay One-Hot Encoding con categorías que nunca se presentan en train o tienen representación redundante.

Se incluyen todas las dummies de una variable categórica (no se hace drop='first').

Hay columnas que son copias exactas o que sumadas a otras siempre dan lo mismo (dummy trap).

💡 Conclusiones prácticas:

Tu OneHotEncoder probablemente está generando columnas completamente redundantes en algunas categorías (las de f72_2 parecen ser un caso de codificación con muchísimas categorías).

Esto explica por qué Linear Regression explotaba** → la matriz X'X es singular o casi singular.

Ridge puede estabilizar algo, pero si el VIF es inf es porque tienes redundancia total; lo ideal sería reducir esas columnas.

Dado que ya usas drop='first' en el OneHotEncoder, podría estar pasando que algunas categorías están 100% correlacionadas con otras variables (por ejemplo, dummies para ubicaciones que coinciden con otra variable geográfica).`

In [ ]:
# =========================================
# 2) Entrenar Ridge Regression
# =========================================

from sklearn.linear_model import Ridge

modelo_ridge = Ridge(alpha=10)  # alpha controla regularización; probar 1, 10, 100
modelo_ridge.fit(X_train_proc, y_train)

y_pred = modelo_ridge.predict(X_test_proc)

# Evaluación
mae_log = mean_absolute_error(y_test, y_pred)
mse_log = mean_squared_error(y_test, y_pred)
rmse_log = np.sqrt(mse_log)
r2_log = r2_score(y_test, y_pred)

# Invertimos escala log para métricas en pesos
y_pred_original = np.exp(y_pred)
y_test_original = np.exp(y_test)

mae_pesos = mean_absolute_error(y_test_original, y_pred_original)
rmse_pesos = np.sqrt(mean_squared_error(y_test_original, y_pred_original))

print("\n✅ Métricas Ridge Regression:")
print(f"R2: {r2_log:.4f} | MAE_log: {mae_log:.4f} | RMSE_log: {rmse_log:.4f}")
print(f"MAE_pesos: {mae_pesos:.2f} | RMSE_pesos: {rmse_pesos:.2f}")

R² = 0.8284: 
- Esto significa que el modelo está explicando más del 82% de la variabilidad en la escala logarítmica, lo cual es muy bueno para un dataset socioeconómico con alta dispersión.
- A diferencia de la regresión lineal simple (que estaba totalmente inestable), el Ridge está controlando la multicolinealidad y manteniendo los coeficientes en rangos razonables.

Errores bajos en log:
- Un MAE_log = 0.2006 y un RMSE_log = 0.2770 son bastante competitivos.
- En términos prácticos, el error relativo promedio es de aproximadamente un 20% en escala log, lo cual es aceptable.

Errores en pesos realistas:
- MAE_pesos ≈ 56 mil pesos y RMSE_pesos ≈ 3.4 M indican que, aunque hay outliers que generan error grande en RMSE, la predicción media está razonablemente cerca de los valores reales.
- Este tipo de error es muchísimo menor que el que vimos en la lineal clásica.

Estabilidad:
- No se observan predicciones absurdas como en la lineal estándar.
- Esto significa que la regularización está funcionando y el modelo no está explotando con valores fuera de rango.

💡 Interpretación
La regularización de Ridge está funcionando:
Penaliza los coeficientes grandes, estabiliza el sistema y permite que la regresión se ajuste razonablemente pese a la alta correlación entre features.

Persisten outliers importantes:
El salto grande de MAE a RMSE en pesos sugiere que unas pocas observaciones extremas están penalizando bastante el RMSE.

El VIF infinito ya no destroza el modelo gracias a la regularización, pero eliminar redundancias aún podría mejorar estabilidad y reducir dimensionalidad.

Ridge es una opción sólida como baseline lineal antes de pasar a modelos más complejos como Random Forest o Gradient Boosting.

In [ ]:
# 4) Modelo 2: Random Forest
# (hiperparámetros base; ajustamos luego)

modelo_rf = RandomForestRegressor(
    n_estimators=400,
    max_depth=None,
    min_samples_leaf=1,
    random_state=42,
    n_jobs=-1
)
res_rf = evaluar_modelo(modelo_rf, X_train_proc, y_train, X_test_proc, y_test, nombre="Random Forest")

Interpretación en escala log:
- R² = 0.9398 → Explicamos ~94% de la varianza de los ingresos log-transformados en el set de test. Esto es muy alto y superior al Ridge (0.8284), lo que indica que el modelo captura mejor las relaciones no lineales y posibles interacciones.
- MAE_log = 0.09 y RMSE_log = 0.16 → Errores pequeños en escala log, lo que implica buena capacidad predictiva incluso en datos no vistos.

Interpretación en pesos:
- MAE_pesos ≈ 11.4 mil pesos → Error medio absoluto relativamente bajo considerando que estamos modelando ingresos que pueden ir desde cero hasta más de un millón de pesos.
- RMSE_pesos ≈ 33.6 mil pesos → El error cuadrático medio también es bajo, aunque mayor que el MAE, lo que indica que aún hay algunos errores grandes en casos puntuales (probablemente en los ingresos más altos).

Conclusión:
- Muy prometedor para seguir con XGBoost y LightGBM.
- Este modelo ya es útil para predicción práctica, pero podemos intentar afinarlo con RandomizedSearchCV para mejorar aún más.
- Posible riesgo: podría haber algo de sobreajuste si la validación no es lo suficientemente estricta (RF tiende a eso con muchos árboles y profundidad alta).

Auditoría de features (Feature Importance)

Esto te permite ver cuáles son las variables más influyentes según el modelo. Si aparece una variable que contiene información directa del ingreso (por ejemplo, otro tipo de ingreso o una suma de ingresos del hogar), entonces es probable que haya fuga de información.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Obtener nombres de las columnas procesadas (después del pipeline)
# Esto asume que hiciste fit en el pipeline con X_train
feature_names = preprocessor.get_feature_names_out()
importancias = modelo_rf.feature_importances_

# Crear DataFrame con los resultados
importancias_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importancias
}).sort_values(by='importance', ascending=False)

# Mostrar las 20 más importantes
print(importancias_df.head(20))

# Graficar
plt.figure(figsize=(10,6))
plt.barh(importancias_df.head(20)['feature'][::-1], importancias_df.head(20)['importance'][::-1])
plt.xlabel("Importancia")
plt.title("Top 20 variables más importantes - Random Forest")
plt.tight_layout()
plt.show()

Observamos:
- La variable num__YDA aporta el 99.8% de la importancia del modelo.
- El resto de las variables tienen importancias casi nulas (valores del orden de 0.001 o incluso 0.000002).
- Entre las más importantes figuran también HT11, YDA_SVL y YSVL, todas ellas potencialmente relacionadas con ingreso.

Interpretación:

- Esto es una señal clara de fuga de información.
- YDA es tu variable objetivo (aunque aquí en su versión logarítmica), y si aparece como predictor, entonces el modelo está simplemente "viendo" la respuesta, lo cual hace inútiles las métricas obtenidas.
- Variables como YDA_SVL o YSVL también suenan sospechosamente similares, por lo que es probable que sean funciones directas o indirectas del ingreso.

Conclusión de esta parte:

- Tu modelo está dominado por variables que contienen directamente (o de forma colineal) la variable que intentás predecir. 
- Esto invalida los resultados. Se debe excluir YDA y toda variable derivada del conjunto de predictores.

Random Forest con Validación cruzada (Cross-Validation)

Esto te dice si el rendimiento es estable entre distintas particiones de los datos. Si hay sobreajuste, el resultado promedio será mucho más bajo que el obtenido en tu partición test anterior.

In [ ]:
from sklearn.model_selection import cross_val_score

# Pipeline que combina preprocesamiento y modelo
modelo_rf_cv = Pipeline(steps=[
    ('preprocesamiento', preprocessor),
    ('modelo', RandomForestRegressor(random_state=42))
])

# Ejecutar CV con el pipeline
scores = cross_val_score(modelo_rf_cv, X, y, cv=5, scoring='r2')

print("R² por fold:", scores)
print(f"Media R²: {scores.mean():.4f}")
print(f"Desviación estándar: {scores.std():.4f}")

Observamos:

Nivel de rendimiento:
- Media R² ≈ 0.8576 significa que, en promedio, el modelo explica alrededor del 85,8% de la variabilidad del ingreso en los datos de validación.
- Esto es menor al R² ≈ 0.94 que obtuviste en el train/test split único, lo que es normal porque la validación cruzada da una estimación más realista y menos optimista.

Estabilidad del modelo:
- Desviación estándar: 0.0127 → muy baja, lo que indica que el rendimiento es consistente entre folds y que el modelo es estable ante cambios en el conjunto de entrenamiento.
- Un desvío bajo sugiere que el modelo no depende demasiado de un subconjunto particular de los datos.

Posible sobreajuste:
- La diferencia entre el R² alto en train/test único y el R² de CV sugiere que el modelo podría estar sobreajustando levemente.
- Esto se puede trabajar ajustando hiperparámetros (max_depth, min_samples_split, etc.) o reduciendo dimensionalidad.

Conclusión:
- El modelo es fuerte y estable, pero el gap entre test único y CV sugiere que todavía hay margen para mejorar generalización.
- Ya está en un rango de rendimiento alto para este tipo de problema, así que es un buen candidato para afinar con GridSearch o RandomizedSearch.

In [ ]:
# 3) XGBoost (Modelo 3)

try:
    from xgboost import XGBRegressor

    modelo_xgb = XGBRegressor(
        n_estimators=800,
        max_depth=8,
        learning_rate=0.05,
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        reg_alpha=0.0,
        random_state=42,
        tree_method="hist",     # rápido y estable
        n_jobs=-1
    )
    res_xgb = evaluar_modelo(modelo_xgb, X_train_proc, y_train, X_test_proc, y_test, nombre="XGBoost")

except ImportError:
    print("\n[AVISO] xgboost no está instalado. Ejecuta: pip install xgboost")

Comparación con Random Forest (que reportaste antes)
- Random Forest: R² = 0.9398 | MAE_pesos ≈ 11 394 | RMSE_pesos ≈ 33 638
- XGBoost: R² = 0.9335 | MAE_pesos ≈ 13 736 | RMSE_pesos ≈ 32 255

Lectura de los resultados:
- R²: Muy parecidos, aunque el RF todavía tiene una leve ventaja en tu dataset.
- Error absoluto medio (MAE): El XGBoost tiene un MAE en pesos un poco más alto que el RF (≈ +2 300 pesos), lo que indica que, en promedio, sus predicciones se desvían un poco más del valor real.
- Error cuadrático medio (RMSE): Aquí el XGBoost gana levemente (32 255 vs 33 638), lo que sugiere que maneja mejor los casos extremos y reduce el impacto de los outliers en el error promedio cuadrático.

Conclusión:
- Ambos modelos son fuertes candidatos y están muy parejos.
- El RF parece un poco más preciso en predicciones promedio (mejor MAE), mientras que XGBoost parece algo más robusto con valores extremos (mejor RMSE).
- Esto es exactamente el tipo de escenario donde hacer cross-validation en igualdad de condiciones para ambos y luego un tuning de hiperparámetros puede inclinar la balanza.

In [ ]:
# 4) LightGBM (Modelo 4)

try:
    from lightgbm import LGBMRegressor

    modelo_lgb = LGBMRegressor(
        n_estimators=1200,
        learning_rate=0.05,
        num_leaves=63,          # ~ 2^(max_depth) aprox (si max_depth ~ 6)
        max_depth=-1,           # sin límite (usa num_leaves como control)
        subsample=0.8,
        colsample_bytree=0.8,
        reg_lambda=1.0,
        random_state=42,
        n_jobs=-1
    )
    res_lgb = evaluar_modelo(modelo_lgb, X_train_proc, y_train, X_test_proc, y_test, nombre="LightGBM")

except ImportError:
    print("\n[AVISO] lightgbm no está instalado. Ejecuta: pip install lightgbm")

Lectura de los resultados:
- R²: RF sigue siendo el que más explica la varianza. LGBM está apenas 0.0032 puntos por debajo de RF y mejora a XGB.
- MAE_pesos (error promedio): RF sigue ganando en error absoluto promedio. LGBM mejora a XGB en MAE, pero todavía no alcanza a RF.
- RMSE_pesos (penaliza outliers): Aquí LGBM es el mejor de los tres, lo que indica que maneja muy bien valores extremos y reduce el impacto de grandes desviaciones.

Conclusiones:
- RF = mejor en precisión media (MAE más bajo).
- LGBM = mejor en robustez frente a valores extremos (RMSE más bajo).
- XGB = intermedio, pero queda por debajo de LGBM en casi todo.
- Te importa predecir con menor error promedio, RF afinado es el candidato.
- Te importa evitar grandes errores puntuales, LGBM afinado podría ser la elección.

Random Forest con GridSearchCV explota la memoria.

In [ ]:
# 5) Random Forest con GridSearchCV (Modelo 5)

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

# Búsqueda en una grilla compacta pero informativa
param_grid_rf = {
    "n_estimators": [300, 500, 800],
    "max_depth": [None, 12, 20],
    "min_samples_split": [2, 10],
    "min_samples_leaf": [1, 2, 4],
    "max_features": ["sqrt", "log2"]
}

rf_base = RandomForestRegressor(random_state=42, n_jobs=-1)

# Usamos RMSE negativo como scoring (minimizar RMSE)
grid_rf = GridSearchCV(
    estimator=rf_base,
    param_grid=param_grid_rf,
    scoring="neg_root_mean_squared_error",
    cv=3,
    n_jobs=-1,
    verbose=1
)

# Ajuste con TRAIN ya preprocesado
grid_rf.fit(X_train_proc, y_train)

print("\n=== GridSearchCV RandomForest ===")
print("Mejores hiperparámetros:", grid_rf.best_params_)
print(f"Mejor score CV (RMSE): {-grid_rf.best_score_:,.4f}")

# Evaluación final en TEST con el mejor estimador
mejor_rf = grid_rf.best_estimator_
res_rf_cv = evaluar_modelo(mejor_rf, X_train_proc, y_train, X_test_proc, y_test, nombre="Random Forest (GridSearchCV)")

Recordamos:
- Los árboles/boosting suelen rendir mejor que los lineales en este tipo de datos tabulares con alta no linealidad e interacciones.
- Si el tiempo es un factor, podés reducir la grilla del RF (menos combinaciones) o usar RandomizedSearchCV.
- Si tu y es log_YDA, las métricas “originales” que imprime el bloque (deslogarizando) son las más interpretables para presentación

In [ ]:
# RandomizedSearchCV XGBoost

#import numpy as np
#import pandas as pd
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import make_scorer, mean_squared_error

# Scoring de RMSE (negativo para que lo maximice)
scoring_rmse = "neg_root_mean_squared_error"

try:
    from xgboost import XGBRegressor
    from scipy.stats import randint, uniform, loguniform

    xgb_base = XGBRegressor(
        random_state=42,
        n_jobs=-1,
        tree_method="hist"   # rápido y estable
    )

    dist_xgb = {
        "n_estimators": randint(400, 1500),
        "max_depth": randint(3, 12),
        "learning_rate": loguniform(1e-3, 3e-1),
        "subsample": uniform(0.6, 0.4),         # [0.6, 1.0]
        "colsample_bytree": uniform(0.6, 0.4),  # [0.6, 1.0]
        "reg_lambda": loguniform(1e-3, 10),
        "reg_alpha": loguniform(1e-4, 1)
    }

    rnd_xgb = RandomizedSearchCV(
        estimator=xgb_base,
        param_distributions=dist_xgb,
        n_iter=40,              # podés subir/bajar según tiempo
        scoring=scoring_rmse,
        cv=3,
        n_jobs=-1,
        random_state=42,
        verbose=1
    )

    rnd_xgb.fit(X_train_proc, y_train)
    print("\n=== RandomizedSearchCV XGBoost ===")
    print("Mejores hiperparámetros:", rnd_xgb.best_params_)
    print(f"Mejor score CV (RMSE): {-rnd_xgb.best_score_:,.4f}")

    best_xgb = rnd_xgb.best_estimator_
    res_xgb = evaluar_modelo(best_xgb, X_train_proc, y_train, X_test_proc, y_test, nombre="XGBoost (RandomizedSearchCV)")

except ImportError:
    best_xgb = None
    print("\n[AVISO] xgboost no está instalado. Ejecuta: pip install xgboost")

In [ ]:
# RandomizedSearchCV LightGBM

try:
    from lightgbm import LGBMRegressor
    from scipy.stats import randint, uniform, loguniform

    lgb_base = LGBMRegressor(
        random_state=42,
        n_jobs=-1
    )

    dist_lgb = {
        "n_estimators": randint(400, 1500),
        "learning_rate": loguniform(1e-3, 3e-1),
        "num_leaves": randint(31, 255),
        "max_depth": [-1, 6, 8, 10, 12],          # categórico
        "subsample": uniform(0.6, 0.4),
        "colsample_bytree": uniform(0.6, 0.4),
        "reg_lambda": loguniform(1e-3, 10),
        "min_child_samples": randint(5, 100)
    }

    rnd_lgb = RandomizedSearchCV(
        estimator=lgb_base,
        param_distributions=dist_lgb,
        n_iter=40,
        scoring=scoring_rmse,
        cv=3,
        n_jobs=-1,
        random_state=42,
        verbose=1
    )

    rnd_lgb.fit(X_train_proc, y_train)
    print("\n=== RandomizedSearchCV LightGBM ===")
    print("Mejores hiperparámetros:", rnd_lgb.best_params_)
    print(f"Mejor score CV (RMSE): {-rnd_lgb.best_score_:,.4f}")

    best_lgb = rnd_lgb.best_estimator_
    res_lgb = evaluar_modelo(best_lgb, X_train_proc, y_train, X_test_proc, y_test, nombre="LightGBM (RandomizedSearchCV)")

except ImportError:
    best_lgb = None
    print("\n[AVISO] lightgbm no está instalado. Ejecuta: pip install lightgbm")

In [ ]:
# Permutation Importance (sobre el mejor modelo)

from sklearn.inspection import permutation_importance
import matplotlib.pyplot as plt

# 1) Obtener nombres de features del preprocessor ya fiteado
try:
    feature_names = preprocessor.get_feature_names_out(X.columns)
except Exception:
    # fallback: índices si no hay nombres
    feature_names = np.array([f"f{i}" for i in range(X_train_proc.shape[1])])

# 2) Elegir el mejor modelo entre XGB y LGB (por RMSE en test)
candidatos = []
if 'res_xgb' in locals():
    candidatos.append(("XGBoost", best_xgb, res_xgb['rmse']))
if 'res_lgb' in locals():
    candidatos.append(("LightGBM", best_lgb, res_lgb['rmse']))

if candidatos:
    ganador_nombre, ganador_modelo, _ = sorted(candidatos, key=lambda t: t[2])[0]
    print(f"\nModelo ganador para interpretación: {ganador_nombre}")
else:
    # Si no hay XGB/LGB disponibles, podés elegir otro (ej: mejor RF)
    ganador_nombre, ganador_modelo = "Modelo_no_disponible", None

# 3) Permutation importance (si hay modelo)
if ganador_modelo is not None:
    # Para acelerar, usar una muestra del test (opcional)
    np.random.seed(42)
    idx_sample = np.random.choice(X_test_proc.shape[0], size=min(3000, X_test_proc.shape[0]), replace=False)
    X_te_pi = X_test_proc[idx_sample]
    y_te_pi = y_test.iloc[idx_sample] if hasattr(y_test, "iloc") else y_test[idx_sample]

    print("\nCalculando Permutation Importance (esto puede tardar un poco)...")
    pi = permutation_importance(
        estimator=ganador_modelo,
        X=X_te_pi,
        y=y_te_pi,
        n_repeats=5,
        scoring="neg_root_mean_squared_error",
        random_state=42,
        n_jobs=-1
    )

    importancias = pd.DataFrame({
        "feature": feature_names,
        "importance_mean": pi.importances_mean,
        "importance_std": pi.importances_std
    }).sort_values("importance_mean", ascending=False)

    top = importancias.head(20)
    print("\nTop 20 features por Permutation Importance:")
    display(top)

    # Plot
    plt.figure(figsize=(8, 10))
    plt.barh(top["feature"][::-1], top["importance_mean"][::-1])
    plt.title(f"Permutation Importance - Top 20 ({ganador_nombre})")
    plt.xlabel("Mean decrease in score (|neg RMSE|)")
    plt.tight_layout()
    plt.show()
else:
    print("\n[AVISO] No hay modelo ganador para calcular permutation importance.")


In [ ]:
# SHAP para el modelo ganador

try:
    import shap

    if ganador_modelo is not None:
        # Crear DataFrame con nombres de columnas (mejora los gráficos)
        X_te_df = pd.DataFrame(X_test_proc, columns=feature_names)

        # Muestra para acelerar el cómputo
        np.random.seed(42)
        samp = min(2000, X_te_df.shape[0])
        X_te_sample = X_te_df.sample(samp, random_state=42)

        # Explainer para modelos de árboles (XGB/LGB/RF)
        explainer = shap.TreeExplainer(ganador_modelo)
        shap_values = explainer.shap_values(X_te_sample)

        # Resumen global
        shap.summary_plot(shap_values, X_te_sample, show=True, max_display=25)
        # (Opcional) resumen por bar:
        shap.summary_plot(shap_values, X_te_sample, plot_type="bar", show=True, max_display=25)

        # Nota: los SHAP están en la misma escala que el target (log_YDA).
    else:
        print("\n[AVISO] No hay modelo ganador para calcular SHAP.")

except ImportError:
    print("\n[AVISO] shap no está instalado. Ejecuta: pip install shap")

Consideramos:
- Nombres de features: preprocessor.get_feature_names_out(X.columns) te devuelve los nombres ya expandidos por el OneHot; así los gráficos y tablas quedan legibles.
- Tiempo de cómputo: tanto RandomizedSearchCV como SHAP pueden tardar; por eso limito n_iter y tomo muestras de test para importancia/SHAP.
- Interpretación: SHAP y permutation importance reportan contribuciones en escala de y. Como vos usás log_YDA, las contribuciones están en log-ingreso (lo señalás en el informe).

Tabla comparativa de modelos:

In [ ]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

# Función auxiliar para evaluar en log y en escala original (pesos)
def evaluar_modelo_full(nombre, modelo, X_tr_proc, X_te_proc, y_tr, y_te):
    # En log
    y_pred_train = modelo.predict(X_tr_proc)
    y_pred_test  = modelo.predict(X_te_proc)

    mae_log  = mean_absolute_error(y_te, y_pred_test)
    rmse_log = np.sqrt(mean_squared_error(y_te, y_pred_test))
    r2_log   = r2_score(y_te, y_pred_test)

    # A escala original
    y_te_pesos    = np.exp(y_te)
    y_pred_pesos  = np.exp(y_pred_test)

    mae_pesos  = mean_absolute_error(y_te_pesos, y_pred_pesos)
    rmse_pesos = np.sqrt(mean_squared_error(y_te_pesos, y_pred_pesos))
    r2_pesos   = r2_score(y_te_pesos, y_pred_pesos)

    return {
        "Modelo": nombre,
        "MAE_log": mae_log,
        "RMSE_log": rmse_log,
        "R2_log": r2_log,
        "MAE_pesos": mae_pesos,
        "RMSE_pesos": rmse_pesos,
        "R2_pesos": r2_pesos
    }

# Lista de modelos ya entrenados y sus nombres
modelos_resultados = []

# Modelo 1: Regresión lineal
modelos_resultados.append(evaluar_modelo_full("LinearRegression", modelo_lr, X_train_proc, X_test_proc, y_train, y_test))

# Modelo 2: Random Forest básico
modelos_resultados.append(evaluar_modelo_full("RandomForest", modelo_rf, X_train_proc, X_test_proc, y_train, y_test))

# Modelo 3: XGBoost optimizado
modelos_resultados.append(evaluar_modelo_full("XGBoost", best_xgb, X_train_proc, X_test_proc, y_train, y_test))

# Modelo 4: LightGBM optimizado
modelos_resultados.append(evaluar_modelo_full("LightGBM", best_lgb, X_train_proc, X_test_proc, y_train, y_test))

# Modelo 5: Random Forest con GridSearchCV
modelos_resultados.append(evaluar_modelo_full("RandomForest (tuned)", best_rf, X_train_proc, X_test_proc, y_train, y_test))

# Crear tabla resumen
tabla_resultados = pd.DataFrame(modelos_resultados)

# Ordenar por RMSE_log ascendente (mejor modelo arriba)
tabla_resultados = tabla_resultados.sort_values(by="RMSE_log")

# Mostrar con 3 decimales
pd.set_option("display.float_format", lambda x: f"{x:,.3f}")
display(tabla_resultados)

Probamos con RandomizedSearchCV para RF, XGBoost y LightGBM con float32 y cv=3 y comparativo de modelos.

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import numpy as np
import pandas as pd
import time

# =====================
# 1) Preprocesar con float32 para reducir memoria
# =====================
X_train_proc = preprocessor.fit_transform(X_train).astype(np.float32)
X_test_proc  = preprocessor.transform(X_test).astype(np.float32)

# =====================
# 2) Función para evaluación rápida
# =====================
def evaluar_modelo_simple(modelo, X_train, y_train, X_test, y_test, nombre="Modelo"):
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)

    mae_log = mean_absolute_error(y_test, y_pred)
    rmse_log = np.sqrt(mean_squared_error(y_test, y_pred))
    r2 = r2_score(y_test, y_pred)

    # Revertimos log a pesos
    y_test_original = np.exp(y_test)
    y_pred_original = np.exp(y_pred)
    mae_pesos = mean_absolute_error(y_test_original, y_pred_original)
    rmse_pesos = np.sqrt(mean_squared_error(y_test_original, y_pred_original))

    return {
        "Modelo": nombre,
        "R2": r2,
        "MAE_log": mae_log,
        "RMSE_log": rmse_log,
        "MAE_pesos": mae_pesos,
        "RMSE_pesos": rmse_pesos
    }

# =====================
# 3) RandomizedSearchCV para cada modelo
# =====================

# --- Random Forest ---
param_rf = {
    "n_estimators": [200, 400, 600],
    "max_depth": [10, 15, 20, None],
    "min_samples_split": [2, 5, 10],
    "min_samples_leaf": [1, 2, 4]
}
rf = RandomForestRegressor(random_state=42, n_jobs=-1)
search_rf = RandomizedSearchCV(rf, param_rf, n_iter=10, cv=3, scoring='r2',
                               verbose=2, random_state=42, n_jobs=-1)
t0 = time.time()
search_rf.fit(X_train_proc, y_train)
print("Random Forest mejores params:", search_rf.best_params_, "Tiempo:", round(time.time()-t0, 2), "s")

# --- XGBoost ---
param_xgb = {
    "n_estimators": [200, 400, 600],
    "max_depth": [3, 5, 7, 10],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0]
}
xgb = XGBRegressor(objective='reg:squarederror', random_state=42, n_jobs=-1)
search_xgb = RandomizedSearchCV(xgb, param_xgb, n_iter=10, cv=3, scoring='r2',
                                verbose=2, random_state=42, n_jobs=-1)
t0 = time.time()
search_xgb.fit(X_train_proc, y_train)
print("XGBoost mejores params:", search_xgb.best_params_, "Tiempo:", round(time.time()-t0, 2), "s")

# --- LightGBM ---
param_lgb = {
    "n_estimators": [200, 400, 600],
    "max_depth": [-1, 10, 15, 20],
    "learning_rate": [0.01, 0.05, 0.1],
    "subsample": [0.7, 0.8, 1.0],
    "colsample_bytree": [0.7, 0.8, 1.0]
}
lgb = LGBMRegressor(objective='regression', random_state=42, n_jobs=-1)
search_lgb = RandomizedSearchCV(lgb, param_lgb, n_iter=10, cv=3, scoring='r2',
                                verbose=2, random_state=42, n_jobs=-1)
t0 = time.time()
search_lgb.fit(X_train_proc, y_train)
print("LightGBM mejores params:", search_lgb.best_params_, "Tiempo:", round(time.time()-t0, 2), "s")

# =====================
# 4) Evaluar y comparar
# =====================
resultados = []
resultados.append(evaluar_modelo_simple(search_rf.best_estimator_, X_train_proc, y_train, X_test_proc, y_test, "Random Forest"))
resultados.append(evaluar_modelo_simple(search_xgb.best_estimator_, X_train_proc, y_train, X_test_proc, y_test, "XGBoost"))
resultados.append(evaluar_modelo_simple(search_lgb.best_estimator_, X_train_proc, y_train, X_test_proc, y_test, "LightGBM"))

df_resultados = pd.DataFrame(resultados)
print("\n📊 Comparativa de modelos:")
print(df_resultados)

: 